In [46]:
import pandas as pd

# Cargar dataset
data = pd.read_csv("Fifa_WC_2022_Match_data.csv")

# Visualizar las primeras filas
print(data.head())

# Comprobar datos nulos o inconsistencias
print(data.info())

# Normalizar nombres de equipos
data['1'] = data['1'].str.upper().str.strip()
data['2'] = data['2'].str.upper().str.strip()

# Verificar equipos únicos
equipos = list(set(data['1']).union(set(data['2'])))
print(f"Equipos en el torneo: {equipos}")


   match_no day_of_week       date   hour                          venue  \
0         1         Sun  20-Nov-22  17:00                Al Bayt Stadium   
1         2         Mon  21-Nov-22  14:00  Khalifa International Stadium   
2         3         Mon  21-Nov-22  17:00             Al Thumama Stadium   
3         4         Mon  21-Nov-22  20:00          Ahmed bin Ali Stadium   
4         5         Tue  22-Nov-22  11:00          Lusail Iconic Stadium   

                         referee    group              1             2  \
0                 Daniele Orsato  Group A          QATAR       ECUADOR   
1                  Raphael Claus  Group B        ENGLAND          IRAN   
2                 Wilton Sampaio  Group A        SENEGAL   NETHERLANDS   
3  Abdulrahman Ibrahim Al Jassim  Group B  UNITED STATES         WALES   
4                  Slavko Vincic  Group C      ARGENTINA  SAUDI ARABIA   

   attendance  ...  1_panelties_scored  2_panelties_scored  1_goal_prevented  \
0       67372  ...

In [5]:
# Calcular métricas para equipo 1 (local)
local = data.groupby('1').agg({
    '1_goals': 'sum',      # Goles anotados
    '2_goals': 'sum',      # Goles recibidos
    '1_xg': 'mean',        # Promedio xG
    '1_poss': 'mean'       # Promedio de posesión
}).rename(columns={'1_goals': 'goals_for', '2_goals': 'goals_against', '1_xg': 'xg', '1_poss': 'possession'})

# Calcular métricas para equipo 2 (visitante)
visitante = data.groupby('2').agg({
    '2_goals': 'sum',
    '1_goals': 'sum',
    '2_xg': 'mean',
    '2_poss': 'mean'
}).rename(columns={'2_goals': 'goals_for', '1_goals': 'goals_against', '2_xg': 'xg', '2_poss': 'possession'})

# Combinar métricas
stats = local.add(visitante, fill_value=0)  # Suma métricas locales y visitantes
stats['matches'] = data['1'].value_counts() + data['2'].value_counts()  # Partidos jugados
stats['goals_for_per_match'] = stats['goals_for'] / stats['matches']
stats['goals_against_per_match'] = stats['goals_against'] / stats['matches']

print(stats)


                goals_for  goals_against        xg  possession  matches  \
1                                                                         
ARGENTINA              15              8  4.040000  110.200000        7   
AUSTRALIA               4              6  1.166667   71.666667        4   
BELGIUM                 1              2  1.400000  102.000000        3   
BRAZIL                  8              3  4.833333  112.833333        5   
CAMEROON                4              4  2.050000   87.000000        3   
CANADA                  2              7  2.100000   98.000000        3   
COSTA RICA              3             11  1.350000   63.000000        3   
CROATIA                 8              7  2.125000  112.416667        7   
DENMARK                 1              3  2.000000  120.500000        3   
ECUADOR                 4              3  3.050000  112.500000        3   
ENGLAND                13              4  3.975000  126.500000        5   
FRANCE                 16

In [6]:
# Probabilidad de ganar
stats['win_prob'] = stats['goals_for_per_match'] / (stats['goals_for_per_match'] + stats['goals_against_per_match'])

# Probabilidad de empatar (simplificada)
stats['draw_prob'] = 1 - stats['win_prob'] - (stats['goals_against_per_match'] / stats['matches'])

# Probabilidad de perder
stats['lose_prob'] = 1 - stats['win_prob'] - stats['draw_prob']

print(stats[['win_prob', 'draw_prob', 'lose_prob']])


                win_prob  draw_prob  lose_prob
1                                             
ARGENTINA       0.652174   0.184561   0.163265
AUSTRALIA       0.400000   0.225000   0.375000
BELGIUM         0.333333   0.444444   0.222222
BRAZIL          0.727273   0.152727   0.120000
CAMEROON        0.500000   0.055556   0.444444
CANADA          0.222222   0.000000   0.777778
COSTA RICA      0.214286  -0.436508   1.222222
CROATIA         0.533333   0.323810   0.142857
DENMARK         0.250000   0.416667   0.333333
ECUADOR         0.571429   0.095238   0.333333
ENGLAND         0.764706   0.075294   0.160000
FRANCE          0.666667   0.170068   0.163265
GERMANY         0.545455  -0.101010   0.555556
GHANA           0.416667  -0.194444   0.777778
IRAN            0.363636  -0.141414   0.777778
JAPAN           0.555556   0.194444   0.250000
KOREA REPUBLIC  0.384615   0.115385   0.500000
MEXICO          0.400000   0.266667   0.333333
MOROCCO         0.545455   0.352505   0.102041
NETHERLANDS  

In [8]:
# Probabilidad de anotar un gol en un minuto
stats['goal_per_minute'] = stats['xg'] / 90

stats


,goals_for,goals_against,xg,possession,matches,goals_for_per_match,goals_against_per_match,win_prob,draw_prob,lose_prob,goal_per_minute
1,,,,,,,,,,,
ARGENTINA,15,8,4.040000,110.200000,7,2.142857,1.142857,0.652174,0.184561,0.163265,0.044889
AUSTRALIA,4,6,1.166667,71.666667,4,1.000000,1.500000,0.400000,0.225000,0.375000,0.012963
BELGIUM,1,2,1.400000,102.000000,3,0.333333,0.666667,0.333333,0.444444,0.222222,0.015556
BRAZIL,8,3,4.833333,112.833333,5,1.600000,0.600000,0.727273,0.152727,0.120000,0.053704
CAMEROON,4,4,2.050000,87.000000,3,1.333333,1.333333,0.500000,0.055556,0.444444,0.022778
CANADA,2,7,2.100000,98.000000,3,0.666667,2.333333,0.222222,0.000000,0.777778,0.023333
COSTA RICA,3,11,1.350000,63.000000,3,1.000000,3.666667,0.214286,-0.436508,1.222222,0.015000
CROATIA,8,7,2.125000,112.416667,7,1.142857,1.000000,0.533333,0.323810,0.142857,0.023611
DENMARK,1,3,2.000000,120.500000,3,0.333333,1.000000,0.250000,0.416667,0.333333,0.022222


In [47]:
# import numpy as np

# # Lista de equipos
# equipos = list(stats.index)

# # Matriz de probabilidades
# transition_matrix = pd.DataFrame(index=equipos, columns=equipos, dtype=float)

# # Llenar la matriz con probabilidades de ganar (simplificado)
# for equipo1 in equipos:
#     for equipo2 in equipos:
#         if equipo1 != equipo2:
#             prob_ganar = stats.loc[equipo1, 'win_prob']
#             prob_perder = stats.loc[equipo2, 'win_prob']
#             prob_empatar = 1 - (prob_ganar + prob_perder)
#             transition_matrix.loc[equipo1, equipo2] = prob_ganar
#             transition_matrix.loc[equipo2, equipo1] = prob_perder
#             transition_matrix.loc[equipo1, equipo1] = prob_empatar

# print(transition_matrix)


In [62]:
import pandas as pd
import numpy as np

# Datos base
np.random.seed(7)  # Fijar semilla para reproducibilidad
equipos = [
  'SPAIN', 'IRAN', 'MOROCCO',  'CANADA',
  'FRANCE', 'WALES','DENMARK', 'SERBIA', 
  'AUSTRALIA', 'GERMANY', 'UNITED STATES', 'SENEGAL', 
  'SAUDI ARABIA', 'MEXICO', 'JAPAN', 'COSTA RICA', 
  'BRAZIL', 'SWITZERLAND', 'GHANA', 'POLAND', 
  'NETHERLANDS', 'TUNISIA', 'PORTUGAL', 'ARGENTINA',
  'CROATIA', 'ENGLAND', 'QATAR', 'KOREA REPUBLIC', 
  'ECUADOR', 'BELGIUM', 'URUGUAY', 'CAMEROON']

grupos = ["A"] * 4 + ["B"] * 4 + ["C"] * 4 + ["D"] * 4 + ["E"] * 4 + ["F"] * 4 + ["G"] * 4 + ["H"] * 4
base = pd.DataFrame({"PAIS": equipos, "GRUPO": grupos})

# Función para simular un enfrentamiento
def simular_enfrentamiento(equipo1, equipo2, stats, eliminatorias=False):
    prob_ganar1 = stats.loc[equipo1, 'win_prob']
    prob_ganar2 = stats.loc[equipo2, 'win_prob']
    prob_empatar = 1 - (prob_ganar1 + prob_ganar2)

    # Evitar que las probabilidades sean negativas
    prob_ganar1 = max(prob_ganar1, 0)
    prob_ganar2 = max(prob_ganar2, 0)
    prob_empatar = max(prob_empatar, 0)

    # Normalizar las probabilidades
    total = prob_ganar1 + prob_ganar2 + prob_empatar
    prob_ganar1 /= total
    prob_ganar2 /= total
    prob_empatar /= total

    resultado = np.random.choice(
        [equipo1, "empate", equipo2], 
        p=[prob_ganar1, prob_empatar, prob_ganar2]
    )

    if eliminatorias: 
        if (resultado == "empate"):   

            print(prob_ganar1) 
            print(prob_ganar2) 
            prob_ganar11 = prob_ganar1 / (prob_ganar1 + prob_ganar2)
            prob_ganar22 = prob_ganar2 / (prob_ganar1 + prob_ganar2)
            
            print(f"Tiempos extras y penales: {equipo1} vs {equipo2}")        
            resultado = np.random.choice([equipo1, equipo2], p=[prob_ganar11, prob_ganar22])
            print(f"resultado:\n {resultado}")
            

                
    return resultado

# Generar estadísticas ficticias para los equipos
stats = pd.DataFrame({
    "win_prob": np.random.rand(len(base)),
    "lose_prob": np.random.rand(len(base))
}, index=base['PAIS'])
stats['win_prob'] = stats['win_prob'] / stats['win_prob'].sum()

# Simulación
resultados = pd.DataFrame()
for i in range(1000):  # Simular 1000 torneos
    clasificados = {}

    # Fase de grupos
    for grupo_nombre in base['GRUPO'].unique():
        grupo = base[base['GRUPO'] == grupo_nombre].copy()
        grupo['PUNTOS'] = 0

        # Simular enfrentamientos
        for i, j in [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]:
            equipo1, equipo2 = grupo.iloc[i]['PAIS'], grupo.iloc[j]['PAIS']
            resultado = simular_enfrentamiento(equipo1, equipo2, stats)

            if resultado == equipo1:  # Equipo 1 gana
                grupo.loc[grupo['PAIS'] == equipo1, 'PUNTOS'] += 3
            elif resultado == equipo2:  # Equipo 2 gana
                grupo.loc[grupo['PAIS'] == equipo2, 'PUNTOS'] += 3
            else:  # Empate
                grupo.loc[grupo['PAIS'] == equipo1, 'PUNTOS'] += 1
                grupo.loc[grupo['PAIS'] == equipo2, 'PUNTOS'] += 1

        # Clasificar primeros dos del grupo
        grupo = grupo.sort_values(by='PUNTOS', ascending=False).reset_index(drop=True)
        clasificados[grupo_nombre + "1"] = grupo.iloc[0]['PAIS']
        clasificados[grupo_nombre + "2"] = grupo.iloc[1]['PAIS']

    # Fase eliminatoria
    octavos = [
        (clasificados["A1"], clasificados["B2"]),
        (clasificados["C1"], clasificados["D2"]),
        (clasificados["B1"], clasificados["A2"]),
        (clasificados["D1"], clasificados["C2"]),
    ]
    print("CUARTOS DE FINAL \n")
    cuartos = [simular_enfrentamiento(a[0], a[1], stats, eliminatorias=True) for a in octavos]
    print("SEMIFINALES \n")
    semis = [simular_enfrentamiento(cuartos[0], cuartos[1], stats, eliminatorias=True), simular_enfrentamiento(cuartos[2], cuartos[3], stats, eliminatorias=True)]
    print("FINAL \n")
    final = simular_enfrentamiento(semis[0], semis[1], stats, eliminatorias=True)

    # Guardar resultados
    resultados = pd.concat([resultados, pd.DataFrame([{
        "ganador": final,
        "finalista1": semis[0],
        "finalista2": semis[1],
        "cuarto11": cuartos[0],
        "cuarto12": cuartos[1],
        "cuarto21": cuartos[2],
        "cuarto22": cuartos[3]
    }])], ignore_index=True)

# Contar victorias
victorias = resultados.groupby('ganador').size().reset_index(name='victorias')
victorias['porcentaje_victoria'] = victorias['victorias'] / 1000
victorias = victorias.sort_values(by='porcentaje_victoria', ascending=False)



CUARTOS DE FINAL 

0.04492664872560281
0.03344019249502931
Tiempos extras y penales: CANADA vs WALES
resultado:
 WALES
0.01666986073808522
0.004094598035212296
Tiempos extras y penales: AUSTRALIA vs MEXICO
resultado:
 MEXICO
0.06073242031430907
0.048432376135256014
Tiempos extras y penales: FRANCE vs IRAN
resultado:
 FRANCE
0.023656160653660035
0.031042331002023384
Tiempos extras y penales: SAUDI ARABIA vs GERMANY
resultado:
 SAUDI ARABIA
SEMIFINALES 

0.03344019249502931
0.004094598035212296
Tiempos extras y penales: WALES vs MEXICO
resultado:
 WALES
0.06073242031430907
0.023656160653660035
Tiempos extras y penales: FRANCE vs SAUDI ARABIA
resultado:
 SAUDI ARABIA
FINAL 

0.03344019249502931
0.023656160653660035
Tiempos extras y penales: WALES vs SAUDI ARABIA
resultado:
 WALES
CUARTOS DE FINAL 

0.04492664872560281
0.03344019249502931
Tiempos extras y penales: CANADA vs WALES
resultado:
 CANADA
0.01666986073808522
0.004094598035212296
Tiempos extras y penales: AUSTRALIA vs MEXICO
resul

In [63]:
# Mostrar resultados
print("Países que ganan más torneos:")
print(victorias.head(10))
print()
print("Resultados:")
print(resultados[['finalista1', 'finalista2', 'cuarto11', 'cuarto12', 'cuarto21', 'cuarto22']].head(10))


Países que ganan más torneos:
          ganador  victorias  porcentaje_victoria
4          FRANCE        248                0.248
13          WALES        196                0.196
6            IRAN        169                0.169
5         GERMANY         97                0.097
0       AUSTRALIA         67                0.067
10   SAUDI ARABIA         53                0.053
12  UNITED STATES         43                0.043
11        SENEGAL         42                0.042
2      COSTA RICA         28                0.028
3         DENMARK         22                0.022

Resultados:
  finalista1    finalista2 cuarto11      cuarto12 cuarto21      cuarto22
0      WALES  SAUDI ARABIA    WALES        MEXICO   FRANCE  SAUDI ARABIA
1     CANADA        FRANCE   CANADA     AUSTRALIA   FRANCE  SAUDI ARABIA
2      WALES       GERMANY    WALES     AUSTRALIA   FRANCE       GERMANY
3      WALES       GERMANY    WALES  SAUDI ARABIA   FRANCE       GERMANY
4      WALES          IRAN    WALES     AU